In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
import re
from datetime import datetime, timedelta
import pymysql
from selenium.webdriver.support.ui import WebDriverWait

In [18]:
url="https://www.amazon.com/-/zh_TW/s?i=electronics-intl-ship&bbn=16225009011&rh=n%3A541966%2Cn%3A193870011%2Cn%3A17923671011%2Cn%3A284822&dc&page={}&language=zh_TW&qid=1598425548&rnid=17923671011&ref=sr_pg_{}"
options = webdriver.ChromeOptions()
options.add_argument("--disable-notifications")
options.add_argument('headless')

chrome = webdriver.Chrome('./chromedriver')
conn = pymysql.connect(**db_settings)
search_date = datetime.strftime(datetime.now() , '%Y-%m-%d')
# 資料庫設定
db_settings = {
    "host": "127.0.0.1",
    "port": 3307, 
    "user": "root",
    "password": "0531",
    "db": "python",
    "charset": "utf8"
}


In [23]:

tStart = time.time()
for i in range(1,3):
    chrome.get(url.format(i,i))
    #加入台幣的cookie，預設為美金
    chrome.delete_cookie("i18n-prefs")    
    chrome.add_cookie({"name":"i18n-prefs","value": "TWD"})
    try:
        soup = BeautifulSoup(chrome.page_source,'lxml')
        amazon_card=soup.find_all(class_="sg-col-4-of-12 sg-col-8-of-16 sg-col-12-of-20 sg-col")
        if len(amazon_card) == 0:
            amazon_card = soup.find_all(class_="s-expand-height s-include-content-margin s-border-bottom s-latency-cf-section")
        with conn.cursor() as cursor:
            for datas in amazon_card:                
                try:
                    brand_item = datas.find('span',{'class':'a-size-medium a-color-base a-text-normal'}).text        
                except:
                    brand_item = datas.find('span',{'class':'a-size-base-plus a-color-base a-text-normal'}).text 
                try:
                    price = int(datas.find('span',{'class':'a-price-whole'}).text.strip('.'))
                except:
                    price = 0                   
                matchObj = re.split(r'\s', brand_item, re.M|re.I)    
                insert = "INSERT INTO `price_comparison` (`brand`, `brand_item`,`price`, `page`, `search_date`)"
                insert += " VALUES ( %s, %s,%s,%s,%s);"
                cursor.execute(insert,(matchObj[0],brand_item,price,i,search_date))
#                 print(insert%(matchObj[0],brand_item,price,i,search_date))
    except Exception as ex:
        print(ex)
conn.commit() 
tEnd = time.time()
print("總共耗時:%f 秒" % (tEnd-tStart))

INSERT INTO `price_comparison` (`brand`, `brand_item`,`price`, `page`, `search_date`) VALUES ( MSI, MSI Gaming Radeon RX GDRR5 DirectX 12 VR Ready CFX 顯示卡,0,1,2020-12-29);
INSERT INTO `price_comparison` (`brand`, `brand_item`,`price`, `page`, `search_date`) VALUES ( Gigabyte, Gigabyte 技嘉 Radeon RX 580 電競 8GB 顯示卡 GV-RX580GAMING-8GD REV2.0,259,1,2020-12-29);
INSERT INTO `price_comparison` (`brand`, `brand_item`,`price`, `page`, `search_date`) VALUES ( MSI微星, MSI微星 Gaming GeForce RTX 3070 8GB GDRR6 256-Bit HDMI/DP TORX 風扇 3.0 Ampere Architecture OC 顯示卡(RTX 3070 Ventus 3X OC),0,1,2020-12-29);
INSERT INTO `price_comparison` (`brand`, `brand_item`,`price`, `page`, `search_date`) VALUES ( MSI微星, MSI微星 Gaming GeForce GT 710 2GB GDRR3 64-bit HDCP 支援 DirectX 12 OpenGL 4.5 單風扇低調顯卡(GT 710 2GD3 LP),51,1,2020-12-29);
INSERT INTO `price_comparison` (`brand`, `brand_item`,`price`, `page`, `search_date`) VALUES ( Gigabyte, Gigabyte 技嘉 Geforce GTX 1050 Ti 顯示卡 內建 4GB 記憶體 搭載風之力風扇 繪圖晶片（GV-N105TWF2OC-4GD）,1

In [ ]:
conn.commit() 
conn.close()
cursor.close()
chrome.close() 

In [54]:
search_date = datetime.strftime(datetime.now() , '%Y-%m-%d')
    # 建立Cursor物件
    with conn.cursor() as cursor:
        for i in amazon_card:
            try:
                brand_item = i.find('span',{'class':'a-size-medium a-color-base a-text-normal'}).text        
            except:
                brand_item = i.find('span',{'class':'a-size-base-plus a-color-base a-text-normal'}).text 
            try:
                price = int(i.find('span',{'class':'a-price-whole'}).text.strip('.'))
            except:
                price = 0
            
            matchObj = re.split(r'\s', brand_item, re.M|re.I)    
            insert = "INSERT INTO `price_comparison` (`brand`, `brand_item`,`price`, `page`, `search_date`)"
            insert += " VALUES ( %s, %s,%s,%s,%s);"

            cursor.execute(insert,(matchObj[0],brand_item,price,1,search_date))
            
        conn.commit() 
        conn.close()
        cursor.close()
except Exception as ex:
    print(ex)